In [1]:
from pynq import Overlay
from pynq import allocate
import time
import numpy as np

In [2]:
# Load Overlay
overlay = Overlay('./cnna.bit')

In [3]:
overlay?

In [4]:
test_ip = overlay.mult_acc

```c
// control
// 0x00 : reserved
// 0x04 : reserved
// 0x08 : reserved
// 0x0c : reserved
// 0x30 : Data signal of out_data_i
//        bit 31~0 - out_data_i[31:0] (Read/Write)
// 0x34 : reserved
// 0x38 : Data signal of out_data_o
//        bit 31~0 - out_data_o[31:0] (Read)
// 0x3c : Control signal of out_data_o
//        bit 0  - out_data_o_ap_vld (Read/COR)
//        others - reserved
// 0x10 ~
// 0x1f : Memory 'in_data' (9 * 8b)
//        Word n : bit [ 7: 0] - in_data[4n]
//                 bit [15: 8] - in_data[4n+1]
//                 bit [23:16] - in_data[4n+2]
//                 bit [31:24] - in_data[4n+3]
// 0x20 ~
// 0x2f : Memory 'in_weights' (9 * 8b)
//        Word n : bit [ 7: 0] - in_weights[4n]
//                 bit [15: 8] - in_weights[4n+1]
//                 bit [23:16] - in_weights[4n+2]
//                 bit [31:24] - in_weights[4n+3]
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)
```

In [10]:
in1 = allocate(shape=(12,), dtype='int8')
in2 = allocate(shape=(12,), dtype='int8')

In [11]:
inarr1 = [1,2,3,4,5,6,7,8,12]
inarr2 = [2,2,2,3,3,3,4,4,4]
for i in range(9):
    in1[i] = inarr1[i]
    in2[i] = inarr2[i]
    
print(in1.shape, in2.shape)
print(in1, in2)
print(in1[0:4].tobytes())
print(type(in1[0].tobytes()))

(12,) (12,)
[ 1  2  3  4  5  6  7  8 12  0  0  0] [2 2 2 3 3 3 4 4 4 0 0 0]
b'\x01\x02\x03\x04'
<class 'bytes'>


In [12]:
test_ip.register_map

RegisterMap {
  out_data_i = Register(out_data_i=0),
  out_data_o = Register(out_data_o=117),
  out_data_o_ctrl = Register(out_data_o_ap_vld=1, RESERVED=0),
  Memory_in_data = Register(value=67305985),
  Memory_in_weights = Register(value=50463234)
}

In [17]:
in1s = 0x10
in2s = 0x20

t1 = time.time()


i = 0
while i < 9:
    if(i%4 == 0):
        print(">at offset",hex(i))
        if(i+4 < in1.shape[0]):
            tdat = in1[i:i+4].tobytes()
        else:
            tdat = in1[i:].tobytes()
            
        print(tdat)
        test_ip.write(in1s+i, tdat)
        
        if(i+4 < in1.shape[0]):
            tdat = in2[i:i+4].tobytes()
        else:
            tdat = in2[i:].tobytes()
            
        print(tdat)
        test_ip.write(in2s+i, tdat)
    i += 1

#test_ip.register_map.Memory_in_data = in1
#test_ip.register_map.Memory_in_weights = in2

#test_ip.write(in1s, in1.tobytes())
#test_ip.write(in2s, in2.tobytes())

t2 = time.time()
print(t2-t1)
print("out:", test_ip.register_map.out_data_i)

>at offset 0x0
b'\x01\x02\x03\x04'
b'\x02\x02\x02\x03'
>at offset 0x4
b'\x05\x06\x07\x08'
b'\x03\x03\x04\x04'
>at offset 0x8
b'\x0c\x00\x00\x00'
b'\x04\x00\x00\x00'
0.005174875259399414
out: 0x0
